In [1]:
import numpy as np
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
import preproc
from importlib import reload

DATA_DIR = "data"

Using TensorFlow backend.


# Prepare training data

In [8]:
VOCAB_SIZE = 3000
data, labels = pickle.load(open(DATA_DIR + "/train_data.pyObj", "rb"))
labels = labels - 1

np.random.seed(125655)
rand_idx = np.random.permutation(range(len(labels)))
train_idx = rand_idx[:int(len(rand_idx) * 0.6)]
cv_idx = rand_idx[int(len(rand_idx) * 0.6):int(len(rand_idx) * 0.2)]
test_idx = rand_idx[int(len(rand_idx) * 0.8):]
text_mat = preproc.preproc_text_vec(data["text"], VOCAB_SIZE)

# Setup Model

In [4]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, 100))
model.add(LSTM(40))
model.add(Dense(6, activation="softmax"))

model.compile(optimizer = 'rmsprop', 
              loss="categorical_crossentropy", 
              metrics = ["accuracy"])


# Training

In [11]:
counter = 0
cost_vec = []
acc_vec = []
for sample_idx in train_idx:
    sample = text_mat[sample_idx]
    label = labels[sample_idx]
    ret = model.train_on_batch(np.array([sample]), keras.utils.to_categorical(label, 6))
    cost_vec.append(ret[0])
    acc_vec.append(ret[1])

    print("[% 4d] cost: %s, accuracy: %.2f" % \
        (counter, np.mean(cost_vec), np.mean(acc_vec)))   
    counter += 1
    if counter > len(train_idx):
        break   

[   0] cost: 1.64583, accuracy: 1.00
[   1] cost: 1.64656, accuracy: 1.00
[   2] cost: 1.69044, accuracy: 0.67
[   3] cost: 1.67622, accuracy: 0.75
[   4] cost: 1.68304, accuracy: 0.60
[   5] cost: 1.70815, accuracy: 0.50
[   6] cost: 1.72724, accuracy: 0.43
[   7] cost: 1.72368, accuracy: 0.50
[   8] cost: 1.73494, accuracy: 0.44
[   9] cost: 1.74006, accuracy: 0.40
[  10] cost: 1.7441, accuracy: 0.36
[  11] cost: 1.75272, accuracy: 0.33
[  12] cost: 1.76333, accuracy: 0.31
[  13] cost: 1.76784, accuracy: 0.29
[  14] cost: 1.77091, accuracy: 0.27
[  15] cost: 1.77195, accuracy: 0.25
[  16] cost: 1.76627, accuracy: 0.29
[  17] cost: 1.76585, accuracy: 0.28
[  18] cost: 1.76953, accuracy: 0.26
[  19] cost: 1.76323, accuracy: 0.30
[  20] cost: 1.76592, accuracy: 0.29
[  21] cost: 1.76761, accuracy: 0.27
[  22] cost: 1.76389, accuracy: 0.30
[  23] cost: 1.76766, accuracy: 0.29
[  24] cost: 1.76982, accuracy: 0.28
[  25] cost: 1.77282, accuracy: 0.27
[  26] cost: 1.76758, accuracy: 0.30
[ 

InvalidArgumentError: indices[0,187] = 3000 is not in [0, 3000)
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_1/embeddings/read, _arg_embedding_1_input_0_2)]]

Caused by op 'embedding_1/Gather', defined at:
  File "c:\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\python36\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "c:\python36\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "c:\python36\lib\site-package